In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
import tkinter as tk
from tkinter import Scrollbar, Text, Button

In [4]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load model and data
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [5]:

# Clean up sentence by tokenizing and lemmatizing
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [6]:
# Create bag of words array
def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {word}")
    return np.array(bag)


In [7]:
# Predict class
def predict_class(sentence):
    p = bag_of_words(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]
    return return_list

In [8]:
# Get response from intents
def getResponse(ints, intents_json):
    if not ints:
        return "Sorry, I didn't understand that."
    
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])
    return "Sorry, I didn't understand that."


In [9]:
# Send message function for GUI
def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("1.0", tk.END)

    if msg:
        ChatBox.config(state=tk.NORMAL)
        ChatBox.insert(tk.END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12))
    
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        
        ChatBox.insert(tk.END, "Bot: " + res + '\n\n')
        ChatBox.config(state=tk.DISABLED)
        ChatBox.yview(tk.END)


In [10]:
# Create main window
root = tk.Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=False, height=False)

# Create chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial")
ChatBox.config(state=tk.DISABLED)

# Create scrollbar
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

# Create send button
SendButton = Button(root, font=("Verdana", 12, 'bold'), text="Send", width="12", height="5",
                    bd=0, bg="#f9a602", activebackground="#3c9d9b", fg='#000000',
                    command=send)

# Create entry box
EntryBox = Text(root, bd=0, bg="white", width="29", height="5", font="Arial")



In [11]:
# Place all components on the screen
scrollbar.place(x=376, y=6, height=386)
ChatBox.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

# Run the application
root.mainloop()

1/1 [==============================] - 0s 25ms/step
